In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x3b2e6f0
Welcome to JupyROOT 6.30/06


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [5]:
ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetPalette(ROOT.kBird)

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
simple_file = True
# generatorStatus of the e+e- system that is immediately before the collision
# 3 for whizard, 4 for madgraph
last_ep_gs = 4

global df
if not simple_file:
    df = ROOT.RDataFrame("events", (
        "data/gen/sw_sl/eLpL.0.edm4hep.root",
        "data/gen/sw_sl/eLpR.0-6.edm4hep.root",
        "data/gen/sw_sl/eRpR.0.edm4hep.root",
        "data/gen/sw_sl/eRpL.0.edm4hep.root",
        )
    )
else:
    # df = ROOT.RDataFrame("events", "data/gen/simple_whizard/enuqq.edm4hep.root")
    # df = ROOT.RDataFrame("events", "data/gen/simple_whizard_ISR/enuqq.edm4hep.root")
    df = ROOT.RDataFrame("events", "data/gen/simple_whizard_FCC_BES_ISR/enuqq.edm4hep.root")
    # df = ROOT.RDataFrame("events", "data/gen/simple_madgraph/enuqq_100k.edm4hep.root")
    # df = ROOT.RDataFrame("events", "data/gen/simple_madgraph/enuqq_1M.edm4hep.root")

#don't calculate weights for simple file
calc_weights = not simple_file
# output_file = "data/stage1/test/sw_sl/nano.root"
# output_file = "data/stage1/test/sw_sl/simple_whizard_nano.root" if simple_file else "data/stage1/test/sw_sl/nano.root"
# output_file = "data/stage1/test/sw_sl/simple_whizard_ISR_nano.root" if simple_file else "data/stage1/test/sw_sl/nano.root"
output_file = "data/stage1/test/sw_sl/simple_whizard_FCC_BES_ISR_nano.root" if simple_file else "data/stage1/test/sw_sl/nano.root"
# output_file = "data/stage1/test/sw_sl/simple_madgraph_nano.root"

In [8]:
# useless in a notebook
# ROOT.RDF.Experimental.AddProgressBar(df)

In [9]:
df.Describe()

Dataframe from TChain events in file data/gen/simple_whizard_FCC_BES_ISR/enuqq.edm4hep.root

Property                Value
--------                -----
Columns in total           67
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Double_t>                    Dataset
EventHeader.weights_begin               ROOT::VecOps::RV

In [10]:
%%cpp
auto bPol_e = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("eL") ? -1 : 1; };
auto bPol_p = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("pL") ? -1 : 1; };
auto nevents = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) {
    // manually extracted :(, not possible to load from json yet because root in the stack is too old
    if (id.Contains("eLpL")) {
        return 100000;
    } else if (id.Contains("eLpR")) {
        return 700000;
    } else if (id.Contains("eRpR")) {
        return 100000;
    } else if (id.Contains("eRpL")) {
        return 100000;
    } else {
        return 0; //should not happen...
    }
};

In [11]:
%%cpp
using namespace ROOT::VecOps;

In [12]:
# what do I want to have?  Leading (stable) e and nu?
# create mask on MCParticles(and leaves) for stable muons
df = df.Define("stable_l_mask", "abs(MCParticles.PDG) == 11 && MCParticles.generatorStatus == 1")
# construct 4-vectors for those MCParticles
df = df.Define("stable_l_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticles.momentum.x[stable_l_mask], MCParticles.momentum.y[stable_l_mask], MCParticles.momentum.z[stable_l_mask], MCParticles.mass[stable_l_mask])")
# get the index of the energy leading lvec in the subset of stable muons
df = df.Define("leading_l_lvec_idx", "return ArgMax(Map(stable_l_lvec, [](const auto &el) {return el.energy();}));")
# use index to get the lvec for the leading stable muon
df = df.Define("leading_l_lvec", "stable_l_lvec[leading_l_lvec_idx]")
# get the pdg of the leading stable muon by first applying the mask to get all stable muons and then use the index to pick the leading one
df = df.Define("leading_l_PDG", "MCParticles.PDG[stable_l_mask][leading_l_lvec_idx]")

df = df.Define("stable_nu_mask", "abs(MCParticles.PDG) == 12 && MCParticles.generatorStatus == 1")
df = df.Define("stable_nu_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticles.momentum.x[stable_nu_mask], MCParticles.momentum.y[stable_nu_mask], MCParticles.momentum.z[stable_nu_mask], MCParticles.mass[stable_nu_mask])")
df = df.Define("leading_nu_lvec_idx", "return ArgMax(Map(stable_nu_lvec, [](const auto &el) {return el.energy();}));")
df = df.Define("leading_nu_lvec", "stable_nu_lvec[leading_nu_lvec_idx]")
df = df.Define("leading_nu_PDG", "MCParticles.PDG[stable_nu_mask][leading_nu_lvec_idx]")

In [13]:
# after ISR and all that beam e+e- have stat 3 YES, AFTER ISR but my seeable vector can contain isr photons that have actual momentum!!!
# bp -> beam particles
df = df.Define("bp_mask", f"abs(MCParticles.PDG) == 11 && MCParticles.generatorStatus == {last_ep_gs}")
df = df.Define("bp_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticles.momentum.x[bp_mask], MCParticles.momentum.y[bp_mask], MCParticles.momentum.z[bp_mask], MCParticles.mass[bp_mask])")
df = df.Define("collision_lvec", "Sum(bp_lvec, ROOT::Math::PxPyPzMVector())")
df = df.Define("collision_lvec_e", "collision_lvec.energy()")

In [14]:
# a bunch of more definitions to calculate E_miss vector
df = df.Define("MCParticles_isNeutrino", "abs(MCParticles.PDG) == 12 || abs(MCParticles.PDG) == 14 || abs(MCParticles.PDG) == 16")
df = df.Define("stable_non_nu_mask", "!MCParticles_isNeutrino && MCParticles.generatorStatus == 1")
df = df.Define("stable_non_nu_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticles.momentum.x[stable_non_nu_mask], MCParticles.momentum.y[stable_non_nu_mask], MCParticles.momentum.z[stable_non_nu_mask], MCParticles.mass[stable_non_nu_mask])")
# TODO: filter out what goes down the beam pipe like ISR
min_theta = 0.05 # rad, FCC-ee lower lumi cal limit iirc.
df = df.Define("seeable_lvecs", f"return Filter(stable_non_nu_lvec, [](const auto &el) {{return abs(cos(el.Theta())) < cos({min_theta});}});")
df = df.Define("seeable_lvec", "Sum(seeable_lvecs, ROOT::Math::PxPyPzMVector())")
# FIXME: no crossing angle and hardcoded sqrt(s)!!!
df = df.Define("E_miss_lvec", "ROOT::Math::PxPyPzEVector(0, 0, 0, 250) - seeable_lvec")
df = df.Define("E_miss_lvec_BS_corrected", "collision_lvec - seeable_lvec")
# FIXME bremsstrahlung and ISR/FSR are not taken into account!! Not clear if brems even makes it into the montecarlo, depends on RecoMCTruthLink setting iirc. ILD uses default -> off
# FIXME provide lvec for each jet separately
df = df.Define("hadronic_lvec", "seeable_lvec - leading_l_lvec")

In [15]:
# print(df.Display(("collision_lvec_e", "seeable_lvec", "E_miss_lvec", "E_miss_lvec_ISR_corrected", "miss_lvec_ISR_corrected_e", "nu_lvec_e", "leading_nu_lvec"), 50).AsString())

In [16]:
df = df.Alias("l_lvec", "leading_l_lvec")
df = df.Alias("nu_lvec", "leading_nu_lvec")
df = df.Alias("l_pdg", "leading_l_PDG")
df = df.Alias("nu_pdg", "leading_nu_PDG")

In [17]:
df = df.Define("l_lvec_e", "l_lvec.energy()")
df = df.Define("nu_lvec_e", "nu_lvec.energy()")
df = df.Define("miss_lvec_e", "E_miss_lvec.energy()")
df = df.Define("miss_lvec_BS_corrected_e", "E_miss_lvec_BS_corrected.energy()")

In [18]:
df = df.Define("nu_diff_e", "nu_lvec_e - miss_lvec_e")
df = df.Define("nu_diff_BS_corrected_e", "nu_lvec_e - miss_lvec_BS_corrected_e")
df = df.Define("miss_lvec_m", "E_miss_lvec.M()")

In [19]:
df = df.Define("xsec_fb", "*PARAMETERS.get<float>(\"crossSection\")")
# FIXME: also check for beamPDG
df = df.Define("beamPol_e", "*PARAMETERS.get<float>(\"beamPol1\")")
df = df.Define("beamPol_p", "*PARAMETERS.get<float>(\"beamPol2\")")
if calc_weights:
    # TODO: obtain beam pol from event parameter
    # df = df.DefinePerSample("beamPol_e", "bPol_e(rdfslot_, rdfsampleinfo_)")
    # df = df.DefinePerSample("beamPol_p", "bPol_p(rdfslot_, rdfsampleinfo_)")
    df = df.DefinePerSample("n_events", "nevents(rdfslot_, rdfsampleinfo_)")

    # df = df.Define("xsec_fb", "PARAMETERS._floatMap.second[PARAMETERS._floatMap.first == crossSection]")
    # df = df.Define("xsec_fb", "PARAMETERS.getValue<float>(\"crossSection\")")
    # weight calc
    ePol_target = 0.0
    pPol_target = 0.0

    lumi_target = 500. # fb^{-1}

    # polweight = (1 + epol*epol_target)*(1 + ppol*ppol_target)/4.;

    # assume one file per process-polarisation combination
    # - in case of several files per process-polarisation combination, need to add up *beforehand*
    # the numbers of events of all the files belonging to the same process-polarisation combination!
    # weight = polweight * xsection * lumi_target / nevt_per_file[ifile];

    # nevt_per_pol = {}
    # polweight = {}

    # def polfilter(ePol: int, pPol: int) -> str:
    #     return f"beamPol_e == {ePol} && beamPol_p == {pPol}"

    # from itertools import product

    # for ePol, pPol in product([-1, +1], repeat=2):
    #     # FIXME: needed for more than one file per pol! also needs summing of the crosssections then!
    #     # nevt_per_pol[(ePol, pPol)] = df.Filter(polfilter(ePol, pPol)).Count()
    #     nevt_per_pol[(ePol, pPol)] = 100000
    #     polweight[(ePol, pPol)] = (1 + ePol * ePol_target) * (1 + pPol * pPol_target) / 4

    df = df.Define("weight", f"1./4. * xsec_fb * {lumi_target} / (double) n_events")
else:
    df = df.Define("weight", "1.")

In [20]:
h_2d_lnu_e = df.Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_lvec_e", "nu_lvec_e", "weight")
h_2d_miss_nu_e = df.Histo2D(("", ";E_{miss} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "miss_lvec_e", "nu_lvec_e", "weight")
h_2d_lmiss_e = df.Histo2D(("", ";E_{e} [GeV];E_{miss} [GeV]", 300, 0., 150., 300, 0., 150.), "l_lvec_e", "miss_lvec_e", "weight")

h_nu_diff = df.Histo1D(("", ";E_{nu}-E_{miss} [GeV]", 300, -50., 5.), "nu_diff_e")
h_nu_diff_ISR_corrected = df.Histo1D(("", ";E_{nu}-E_{miss, no BS} [GeV]", 300, -50., 5.), "nu_diff_BS_corrected_e")
h_miss_m = df.Histo1D(("", ";M_{miss} [GeV]", 300, -25., 125.), "miss_lvec_m")
h_2d_miss_m_nu_diff = df.Histo2D(("", ";M_{miss} [GeV];E_{nu}-E_{miss} [GeV]", 300, -25., 125., 300, -50., 0.), "miss_lvec_m", "nu_diff_e", "weight")

# h_n_events = df.Histo1D("n_events")
# h_weight = df.Histo1D("weight")
# h_xsec_fb = df.Histo1D("xsec_fb")

In [21]:
c_2d_lnu_e = ROOT.TCanvas()
h_2d_lnu_e.Draw("colz0")
c_2d_lnu_e.Draw()

c_2d_lmiss_e = ROOT.TCanvas()
h_2d_lmiss_e.Draw("colz0")
c_2d_lmiss_e.Draw()

c_2d_miss_nu_e = ROOT.TCanvas()
h_2d_miss_nu_e.Draw("colz0")
c_2d_miss_nu_e.Draw()

c_nu_diff = ROOT.TCanvas()
h_nu_diff.Draw("hist")
c_nu_diff.Draw()

c_nu_diff_ISR_corrected = ROOT.TCanvas()
h_nu_diff_ISR_corrected.Draw("hist")
c_nu_diff_ISR_corrected.Draw()

# c_miss_m = ROOT.TCanvas()
# h_miss_m.Draw("hist")
# c_miss_m.Draw()

# c_2d_miss_m_nu_diff = ROOT.TCanvas()
# h_2d_miss_m_nu_diff.Draw("colz0")
# c_2d_miss_m_nu_diff.Draw()

# c_n_events = ROOT.TCanvas()
# h_n_events.Draw("hist")
# c_n_events.Draw()

# c_weight = ROOT.TCanvas()
# h_weight.Draw("hist")
# c_weight.Draw()

# c_xsec_fb = ROOT.TCanvas()
# h_xsec_fb.Draw("hist")
# c_xsec_fb.Draw()

In [22]:
# FIXME: for debug
# multi_qq_evts = df.Filter("l_MC_idx > 20").Take["ULong64_t"]("rdfentry_").GetValue()
# print(multi_qq_evts)

In [23]:
branches = ["beamPol_e", "beamPol_p", "xsec_fb", "weight", "l_lvec", "nu_lvec", "l_pdg", "nu_pdg", "hadronic_lvec", "E_miss_lvec", "collision_lvec"]
# df.Snapshot("events", "data/stage1/test/sw_sl/nano.root", ("beamPol_e", "beamPol_p", "xsec_fb", "weight", "l_lvec", "nu_lvec", "l_pdg", "nu_pdg", "lnu_ICN_lvec", "qq_ICN_lvec"))
df.Snapshot("events", output_file, branches)

<cppyy.gbl.ROOT.RDF.RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > object at 0x1122aad0>